# Algoritmo de Luhn

### Preprocessamento

In [9]:
# !pip install spacy
# !python -m spacy download pt_core_news_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
import spacy
import re
import nltk
import string
import heapq

In [10]:
pln = spacy.load('pt_core_news_sm')

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
def preprocessamento(texto):
  stopwords = nltk.corpus.stopwords.words('portuguese')
  texto = texto.lower()
  texto = re.sub(r" +", ' ', texto)
  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)
  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

In [7]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas. 
                    Definem como o estudo de agente artificial com inteligência. 
                    Ciência e engenharia de produzir máquinas com inteligência. 
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [13]:
preprocessamento_lematizacao(texto_original)

'inteligência artificial ser inteligência similar humano máquina \n  definir comer estudar agente artificial inteligência \n  ciência engenhar produzir máquina inteligência \n  resolver problema possuir inteligência \n  relacionar comportamento inteligente \n  construção máquina parir raciocinar \n  aprender erro acerto \n  inteligência artificial ser raciocinar situação cotidiano'

In [20]:
def calcula_nota_sentenca(sentencas, palavras_importantes, distancia):
  notas = []
  indice_sentenca = 0
  for sentenca in [nltk.word_tokenize(sentenca.lower()) for sentenca in sentencas]:
    indice_palavra = []
    for palavra in palavras_importantes:
      try:
        indice_palavra.append(sentenca.index(palavra))
      except ValueError:
        pass
    indice_palavra.sort()
    if len(indice_palavra) == 0:
      continue
    lista_grupos = []
    grupo = [indice_palavra[0]]
    i = 1
    while i < len(indice_palavra):
      if indice_palavra[i] - indice_palavra[i - 1] < distancia:
        grupo.append(indice_palavra[i])
      else:
        lista_grupos.append(grupo[:])
        grupo = [indice_palavra[i]]
      i += 1
    lista_grupos.append(grupo)
    nota_maxima_grupo = 0
    for g in lista_grupos:
      palavras_importantes_no_grupo = len(g)
      total_palavras_no_grupo = g[-1] - g[0] + 1
      nota = 1.0 * palavras_importantes_no_grupo**2 / total_palavras_no_grupo
      if nota > nota_maxima_grupo:
        nota_maxima_grupo = nota
    notas.append((nota_maxima_grupo, indice_sentenca))
    indice_sentenca += 1
  return notas

In [19]:
def sumarizar(texto, top_n_palavras, distancia, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento_lematizacao(sentenca_original) for sentenca_original in sentencas_originais]
  palavras = [palavra.lower() for sentenca in sentencas_formatadas for palavra in nltk.tokenize.word_tokenize(sentenca)]
  frequencia = nltk.FreqDist(palavras)
  top_n_palavras = [palavra[0] for palavra in frequencia.most_common(top_n_palavras)]
  notas_sentencas = calcula_nota_sentenca(sentencas_formatadas, top_n_palavras, distancia)
  melhores_sentencas = heapq.nlargest(quantidade_sentencas, notas_sentencas)
  melhores_sentencas = [sentencas_originais[i] for (nota, i) in melhores_sentencas]
  
  return sentencas_originais, melhores_sentencas, notas_sentencas

### Teste da Função de Sumarização

In [25]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 5, 3, 3)

In [26]:
melhores_sentencas

['Construção de máquinas para raciocinar.',
 'A inteligência artificial é a inteligência similar à humana máquinas.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

### Sumarização de Artigos de Blog usando o Algoritmo de Luhn

In [24]:
from goose3 import Goose
g = Goose()
url = 'https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=inteligencia-artificial-descobre-leis-fisica-ocultas-dados&id=010150211230'
artigo = g.extract(url)

In [29]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5, 3, 5)

In [30]:
melhores_sentencas

['Essas simulações usam modelos matemáticos baseados nas leis da física que são conhecidas.',
 'Pesquisadores das universidades de Kobe e Osaka, no Japão, desenvolveram um sistema de inteligência artificial capaz de extrair equações de movimento de bases de dados observacionais comuns, e criar um modelo que é fiel às leis da física.',
 'Isso poderia ser usado para investigar a sustentabilidade do ecossistema, aplicando as leis físicas apropriadas (por exemplo, a lei de conservação de energia).',
 'Além de fornecer resultados mais rapidamente, serão resultados mais fiéis às leis da física.',
 'As simulações de sistemas físicos são realizadas em uma ampla gama de campos, incluindo previsão do tempo, descoberta de novos medicamentos, análises de edifícios, design de automóveis e muito mais.']